# import

In [12]:
import time
import pandas as pd
from urllib.request import urlopen
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup
import re
import csv

# 出馬表からレース単位で馬IDを抜き出す

In [3]:
def scrape_syutuba(race_id_list):
    data = pd.DataFrame()
    for race_id in tqdm(race_id_list):
        time.sleep(1)
        try:
            url = 'https://race.netkeiba.com/race/shutuba.html?race_id=' + race_id

            html = requests.get(url)
            html.encoding = "EUC-JP"
            soup = BeautifulSoup(html.text, "html.parser")

            # horse_id
            horse_td_list = soup.find_all("td", attrs={'class': 'HorseInfo'})
            dfs = pd.DataFrame()
            for td in horse_td_list:
                horse_id = re.findall(r'\d+', td.find('a')['href'])[0]
                record = pd.Series([horse_id])
                dfs = dfs.append(record,ignore_index=True)

            dfs.index = [race_id] * len(dfs)
            data = data.append(dfs)
        except:
            break

    return data

# 各馬IDから近走成績を取得し、展開バイアスの公式に当てはめてカウントを行う

①近3走以内に全角4番手以内を2回以上通過した事がある馬→1でカウント
②近3走以内に全角4番手以内を1回通過した事がある馬→0.5でカウント
③近3走以内に全角4番手以内を通過した事がない馬→0でカウント（要するにカウントしない）
④逃げ馬カウント（近3走以内に2回以上逃げた馬が1頭もいない場合→－1.5カウント／近3走以内に2回以上逃げた馬が1頭だけの場合→0カウント／近3走以内に2回以上逃げた馬が2頭以上いる場合→＋1.5カウント）

In [4]:
def scrape_horse(horse_id_list):
    #逃げ馬頭数
    nigeuma = 0
    #展開バイアスカウント
    PlcOnCnt = 0.0
    #各馬情報取得
    for horse_id in tqdm(horse_id_list):
        time.sleep(1)
        try:
            url = 'https://db.netkeiba.com/horse/' + horse_id
            df = pd.read_html(url)[3]
            #受賞歴がある馬の場合4番目のデータを取得する
            if df.columns[0]=='受賞歴':
                df = pd.read_html(url)[4]
                
            required_columns = {'通過'}
            if not required_columns <= set(df.columns):
                continue
            
            #各馬近走３レース分のみ取得
            cnt = 0
            #全コーナー４番手以内回数
            Allpas4cnt = 0
            #逃げた回数
            nigecnt = 0
            for passages in df['通過']:
                # 出走取消等は除外
                if type(passages) != str:
                    continue
                # 通過順をリストで取得
                passage = passages.split('-')
                #文字列を数値に変換
                l_passage = [int(s) for s in passage]
                #各コーナーの最大値が４以下であればAll4カウント
                if max(l_passage) <= 4:
                    Allpas4cnt = Allpas4cnt + 1
                #どこかに１があれば逃げたと判定
                if l_passage[0] == 1:
                    nigecnt = nigecnt + 1
                cnt = cnt + 1
                if cnt >= 3:
                    break
            
            #全コーナー４番手以内回数が２以上なら
            if Allpas4cnt >= 2:
                PlcOnCnt = PlcOnCnt + 1
            #全コーナー４番手以内回数が１なら
            elif Allpas4cnt == 1:
                PlcOnCnt = PlcOnCnt + 0.5
            #逃げた回数が２回以上ならカウント
            if nigecnt >= 2:
                nigeuma = nigeuma + 1

        except IndexError:
            continue
        except Exception as e:
            print(e)
            break
        except:
            break

    #逃げ馬の数で最終調整
    if nigeuma == 0:
        PlcOnCnt = PlcOnCnt - 1.5
    elif nigeuma >= 2:
        PlcOnCnt = PlcOnCnt + 1.5

    return PlcOnCnt

# レースIDリスト作成

In [5]:
year = 2021
course_list = [5,9,1]
kai_list = [3,3,1]
day_list = [6,2,4]
racenum = 1
race_id_list = []

for i in range(0,len(course_list),1):
    for race in range(0,racenum,1): 
        race_id = str(year).zfill(4) + str(course_list[i]).zfill(2) + str(kai_list[i]).zfill(2) + str(day_list[i]).zfill(2) +str(race+1).zfill(2)
        race_id_list.append(race_id)

# 出馬表スクレイピング

In [6]:
syutuba_list = scrape_syutuba(race_id_list)

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
syutuba_list

,0
202105030601,2017104880
202105030601,2016104289
202105030601,2015102746
202105030601,2016104963
202105030601,2016104153
202105030601,2014104912
202105030601,2017102786
202105030601,2014100484
202105030601,2016100610
202105030601,2017100925


# 各馬成績スクレイピング　＆　展開バイアス算出

In [8]:
placeonbiases = []
for race_id in race_id_list:
    s_race =  syutuba_list.loc[race_id]
    placecnt = 0.0
    horse_id_list = []
    for ds,row in s_race.iterrows():
        horse_id_list.append(row[0])
    
    # 展開バイアス算出
    print(race_id + '取得中・・・')
    placecnt = scrape_horse(horse_id_list)
    print('カウント完了' + str(placecnt))
    if placecnt > 5:
        placeonbiases.append([race_id,placecnt,'×'])        
    else:
        placeonbiases.append([race_id,placecnt,'〇'])        

202105030601取得中・・・


  0%|          | 0/10 [00:00<?, ?it/s]

カウント完了1.5
202109030201取得中・・・


  0%|          | 0/16 [00:00<?, ?it/s]

カウント完了5.5
202101010401取得中・・・


  0%|          | 0/16 [00:00<?, ?it/s]

カウント完了4.5


In [9]:
placeonbiases

[['202105030601', 1.5, '〇'],
 ['202109030201', 5.5, '×'],
 ['202101010401', 4.5, '〇']]

In [15]:
# listをCSVに出力する
with open('./list.csv','w') as f:
    writer = csv.writer(f)
    for pbias in placeonbiases:
        writer.writerow(pbias)
    
# listをdataframeに変換してからCSVに出力する
pbias_df = pd.DataFrame(placeonbiases)
pbias_df.to_csv('./dataframe.csv', encoding='shift-jis')


# 日本語化

In [32]:
course_dict = {'01':'札幌','02':'函館','03':'福島','04':'新潟','05':'東京','06':'中山','07':'中京','08':'京都','09':'阪神','10':'小倉'}

for placeonbias in placeonbiases:
    rindex = placeonbias[0]
    if placeonbias[1] > 5:
        print(course_dict[rindex[4:6]] + ' ' + rindex[6:8] + '回 ' + rindex[8:10] + '日目 ' + rindex[-2:] + 'レース ' + '展開バイアスなし：カウント' + str(placeonbias[1]))
    else:
        print(course_dict[rindex[4:6]] + ' ' + rindex[6:8] + '回 ' + rindex[8:10] + '日目 ' + rindex[-2:] + 'レース ' + '展開バイアス発生：カウント' + str(placeonbias[1]))

sortlist = sorted(placeonbiases, key=lambda x: x[1])
for placeonbias in sortlist:
    rindex = placeonbias[0]
    if placeonbias[1] > 5:
        print(course_dict[rindex[4:6]] + ' ' + rindex[6:8] + '回 ' + rindex[8:10] + '日目 ' + rindex[-2:] + 'レース ' + '展開バイアスなし：カウント' + str(placeonbias[1]))
    else:
        print(course_dict[rindex[4:6]] + ' ' + rindex[6:8] + '回 ' + rindex[8:10] + '日目 ' + rindex[-2:] + 'レース ' + '展開バイアス発生：カウント' + str(placeonbias[1]))
    

東京 03回 06日目 01レース 展開バイアス発生：カウント1.5
東京 03回 06日目 02レース 展開バイアス発生：カウント0.5
東京 03回 06日目 03レース 展開バイアス発生：カウント3.0
東京 03回 06日目 04レース 展開バイアス発生：カウント0.0
東京 03回 06日目 05レース 展開バイアス発生：カウント-1.5
東京 03回 06日目 06レース 展開バイアス発生：カウント3.0
東京 03回 06日目 07レース 展開バイアスなし：カウント7.5
東京 03回 06日目 08レース 展開バイアス発生：カウント4.0
東京 03回 06日目 09レース 展開バイアス発生：カウント2.0
東京 03回 06日目 10レース 展開バイアスなし：カウント5.5
東京 03回 06日目 11レース 展開バイアスなし：カウント9.5
東京 03回 06日目 12レース 展開バイアスなし：カウント8.5
阪神 03回 02日目 01レース 展開バイアス発生：カウント3.5
阪神 03回 02日目 02レース 展開バイアス発生：カウント5.0
阪神 03回 02日目 03レース 展開バイアス発生：カウント1.0
阪神 03回 02日目 04レース 展開バイアス発生：カウント3.0
阪神 03回 02日目 05レース 展開バイアス発生：カウント-1.5
阪神 03回 02日目 06レース 展開バイアス発生：カウント4.0
阪神 03回 02日目 07レース 展開バイアスなし：カウント6.5
阪神 03回 02日目 08レース 展開バイアス発生：カウント4.0
阪神 03回 02日目 09レース 展開バイアス発生：カウント4.0
阪神 03回 02日目 10レース 展開バイアスなし：カウント6.0
阪神 03回 02日目 11レース 展開バイアス発生：カウント2.0
阪神 03回 02日目 12レース 展開バイアスなし：カウント9.0
札幌 01回 04日目 01レース 展開バイアス発生：カウント4.5
札幌 01回 04日目 02レース 展開バイアス発生：カウント0.5
札幌 01回 04日目 03レース 展開バイアス発生：カウント4.5
札幌 01回 04日目 04レース 展開バイアス発生：カウント1.0
札幌 01回 04日目 05レース 